In [4]:
import requests
from PIL import Image
from transformers import AutoProcessor, Blip2ForConditionalGeneration
import torch
import pandas as pd
from tqdm import tqdm

In [5]:
base_path = ''

file_path = ''
df = pd.read_csv(file_path)
display(df)

,id,country.name,type,imageRelPath,topics,income,tag
0,5d4bea8ccf0b3a0f3f34aa22,India,image,assets/5d4bea8ccf0b3a0f3f34aa22/5d4bea8ccf0b3a...,Get water,29.031387,"animal, barrel, blue, carry, container, cooler..."
1,5d4bea8ccf0b3a0f3f34aa28,India,image,assets/5d4bea8ccf0b3a0f3f34aa28/5d4bea8ccf0b3a...,Chickens,29.031387,"animal, barrel, debris, dirt field, floor, foo..."
2,5d4bea8ccf0b3a0f3f34aa32,India,image,assets/5d4bea8ccf0b3a0f3f34aa32/5d4bea8ccf0b3a...,Fruit trees,29.031387,"palm tree, sky, tree"
3,5d4bea8ccf0b3a0f3f34aa38,India,image,assets/5d4bea8ccf0b3a0f3f34aa38/5d4bea8ccf0b3a...,Photo guide images,29.031387,"camera, catch, smartphone, electronic, hand, p..."
4,5d4bea8dcf0b3a0f3f34aa3a,India,image,assets/5d4bea8dcf0b3a0f3f34aa3a/5d4bea8dcf0b3a...,Bike,29.031387,"attach, bicycle, floor, hay, hut, lean, park, ..."
...,...,...,...,...,...,...,...
6459,5d4be81bcf0b3a0f3f3469d7,China,image,assets/5d4be81bcf0b3a0f3f3469d7/5d4be81bcf0b3a...,TV,10097.721760,"table, drawer, entertainment center, flat, foo..."
6460,5d4be81bcf0b3a0f3f3469db,China,image,assets/5d4be81bcf0b3a0f3f3469db/5d4be81bcf0b3a...,Fruits and vegetables,10097.721760,"banana, biscuit, chip, container, table, dinni..."
6461,5d4be81bcf0b3a0f3f3469df,China,image,assets/5d4be81bcf0b3a0f3f3469df/5d4be81bcf0b3a...,Toys,10097.721760,"animal, doll, greenery, plant, pot, sit, stuff..."
6462,5d4be81bcf0b3a0f3f3469e1,China,image,assets/5d4be81bcf0b3a0f3f3469e1/5d4be81bcf0b3a...,"Vegetable plot,Backyard",10097.721760,"bush, green, grow, ivy, plant, straw hat, sun ..."


In [6]:
processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/127k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/5.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Blip2ForConditionalGeneration(
  (vision_model): Blip2VisionModel(
    (embeddings): Blip2VisionEmbeddings(
      (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (encoder): Blip2Encoder(
      (layers): ModuleList(
        (0-38): 39 x Blip2EncoderLayer(
          (self_attn): Blip2Attention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1408, out_features=4224, bias=True)
            (projection): Linear(in_features=1408, out_features=1408, bias=True)
          )
          (layer_norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
          (mlp): Blip2MLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1408, out_features=6144, bias=True)
            (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          )
          (layer_norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((

In [7]:
def generate_caption(relative_path):
    # Construct the full image path
    full_image_path = os.path.join(base_path, relative_path)
    
    try:
        # Load the image
        with open(full_image_path, 'rb') as f:
            image = Image.open(f).convert('RGB')

        # Process the image and generate a caption
        inputs = processor(images=image, return_tensors="pt").to(device)
        outputs = model.generate(**inputs, max_new_tokens=51)
        caption = processor.decode(outputs[0], skip_special_tokens=True)
        return caption
    except Exception as e:
        print(f"Failed to process image {full_image_path}: {str(e)}")
        return None

In [9]:
df['blip2_caption'] = [generate_caption(path) for path in tqdm(df['imageRelPath'], desc="Generating Captions")]

# Save the DataFrame with captions
output_file_path = '../caption_results/BLIP2_eng_dollarstreet.csv'
df.to_csv(output_file_path, index=False)

print("Captions generated and saved successfully.")

Generating Captions: 100%|██████████| 6464/6464 [55:26<00:00,  1.94it/s]

Captions generated and saved successfully.
